Họ và tên: Đỗ Phương Duy

MSSV: 23520362

In [112]:
# !pip install gymnasium

In [256]:
import gymnasium as gym
import numpy as np
import time
from IPython import display
from gymnasium.envs.toy_text.frozen_lake import FrozenLakeEnv
from gymnasium.envs.toy_text.taxi import TaxiEnv

# FrozenLake 4x4

In [286]:
env = FrozenLakeEnv(desc=None, map_name="4x4", render_mode="ansi", is_slippery=True)
# FrozenLake8x8
# env = FrozenLakeEnv(desc=None, map_name="4x4", render_mode="ansi", is_slippery=True)
# Taxi
# env = TaxiEnv()

In [287]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [288]:
env.observation_space.n

np.int64(16)

In [289]:
env.action_space.n

np.int64(4)

In [118]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [119]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 6)

In [120]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 27)

In [121]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 6)

In [122]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 13)

In [123]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 86)

In [124]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [125]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [126]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 44/1000
Average number of steps: 11.454545454545455


In [127]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 89/1000
Average number of steps: 15.089887640449438


In [128]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 783/1000
Average number of steps: 44.17369093231162


In [290]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [130]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [131]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [132]:
np.all(v_values_1 >= v_values_0)

np.True_

In [133]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [134]:
np.all(v_values_2 >= v_values_1)

np.True_

In [135]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [136]:
np.all(v_values_3 >= v_values_2)

np.True_

In [291]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [292]:
start_time = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end_time = time.time()

Converged at 79-th iteration.


In [293]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [294]:
print(f"Value Iteration took {end_time - start_time:.4f} seconds")


Value Iteration took 0.0250 seconds


In [295]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [187]:
start_time = time.time()

In [188]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [189]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [190]:
end_time = time.time()

In [146]:
play(env, optimal_policy, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 88)

In [147]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 773/1000
Average number of steps: 45.92496765847348


In [296]:
import numpy as np

def policy_iteration(env, max_iters=500, gamma=0.9, tol=1e-6):
    # Khởi tạo chính sách ngẫu nhiên (hành động 0 cho mọi trạng thái)
    policy = np.zeros(env.observation_space.n, dtype=int)
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        # 1. Policy Evaluation: tính giá trị v cho chính sách hiện tại
        v_values = policy_evaluation(env, policy, max_iters=1000, gamma=gamma)

        # 2. Policy Improvement: cập nhật chính sách dựa trên v_values mới
        policy_stable = True

        for state in range(env.observation_space.n):
            old_action = policy[state]
            q_values = []

            for action in range(env.action_space.n):
                q_value = 0
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * v_values[next_state])
                q_values.append(q_value)

            best_action = np.argmax(q_values)
            policy[state] = best_action

            if old_action != best_action:
                policy_stable = False

        if policy_stable:
            print(f'Policy iteration converged at iteration {i}.')
            break

    return policy, v_values


In [297]:
start_time = time.time()
optimal_policy, optimal_p_values = policy_iteration(env, max_iters=500, gamma=0.9)
end_time = time.time()

Converged at 0-th iteration.
Converged at 23-th iteration.
Converged at 59-th iteration.
Converged at 62-th iteration.
Converged at 79-th iteration.
Converged at 80-th iteration.
Policy iteration converged at iteration 5.


In [298]:
print(f"Policy Iteration took {end_time - start_time:.4f} seconds")


Policy Iteration took 0.0174 seconds


In [239]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [240]:
optimal_p_values

array([0.06888666, 0.06141097, 0.07440714, 0.05580443, 0.09185068,
       0.        , 0.11220679, 0.        , 0.14543323, 0.24749485,
       0.29961611, 0.        , 0.        , 0.37993438, 0.63901935,
       0.        ])

In [241]:
play(env, optimal_policy)

(1.0, 77)

In [242]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 757/1000
Average number of steps: 43.849405548216644


## Nhận Xét

**Thời gian chạy:**

Value Iteration: 0.0250 seconds

Policy Iteration: 0.0174 seconds

**Nhận xét**

Ở toy games này thuật toán Policy Iteration chạy nhanh hơn vì có không gian trạng thái nhỏ và 4 hành động cho mỗi trạng thái -> Điều này làm cho bước policy evaluation trong Policy Iteration rất nhanh, vì việc tính toán giá trị cho tất cả trạng thái không tốn nhiều tài nguyên.

Vì môi trường nhỏ, bước đánh giá chính sách (policy evaluation) trong Policy Iteration không mất nhiều thời gian, nên tổng thời gian hội tụ thấp hơn.

# FrozenLake8x8-v1


In [307]:
#FrozenLake8x8
env = FrozenLakeEnv(desc=None, map_name="8x8", render_mode="ansi", is_slippery=True)

In [308]:
env.observation_space.n

np.int64(64)

In [309]:
env.action_space.n

np.int64(4)

In [310]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [311]:
#Value_Iteration
start_time = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_v_values
end_time = time.time()

Converged at 117-th iteration.


In [312]:
print(f"Value Iteration took {end_time - start_time:.4f} seconds")


Value Iteration took 0.1068 seconds


In [313]:
#Policy_Iteration
start_time = time.time()
optimal_p_values = policy_iteration(env, max_iters=500, gamma=0.9)
optimal_p_values
end_time = time.time()

Converged at 27-th iteration.
Converged at 91-th iteration.
Converged at 92-th iteration.
Converged at 86-th iteration.
Converged at 90-th iteration.
Converged at 92-th iteration.
Converged at 95-th iteration.
Converged at 100-th iteration.
Converged at 112-th iteration.
Converged at 117-th iteration.
Policy iteration converged at iteration 9.


In [314]:
print(f"Policy Iteration took {end_time - start_time:.4f} seconds")


Policy Iteration took 0.1402 seconds


## Nhận xét

**Thời gian chạy:**

Value Iteration: 0.1068 seconds

Policy Iteration: 0.1402 seconds

**Nhận xét**

Ở toy games này thuật toán Value Iteration chạy nhanh hơn không đáng kể. Vì với 64 trạng thái, mỗi trạng thái có 4 hành động, không gian chưa quá lớn để bước policy evaluation của Policy Iteration trở nên quá chậm.

Value Iteration cần nhiều vòng hơn nhưng mỗi vòng đơn giản, Policy Iteration cần ít vòng hơn nhưng mỗi vòng tốn công hơn. Ở kích thước này, số vòng và công việc mỗi vòng cân bằng gần bằng, nên thời gian chạy hai thuật toán khá sát nhau.



# Taxi-v3

In [315]:
#Taxi
env = TaxiEnv()

In [316]:
env.observation_space.n

np.int64(500)

In [317]:
env.action_space.n

np.int64(6)

In [318]:
#Value_Iteration
start_time = time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
end_time = time.time()

Converged at 116-th iteration.


In [319]:
optimal_v_values

array([ 89.47323891,  32.81971401,  55.26423891,  37.57755845,
         8.43222921,  32.81971401,   8.43222921,  15.28447953,
        32.81971401,  18.09386122,  55.26423891,  21.2154998 ,
        12.75594298,  18.09386122,  12.75594298,  37.57755845,
       100.52591945,  37.57755845,  62.51591945,  42.86394891,
        79.52591945,  28.53774704,  48.73781945,  32.81971401,
        10.48035311,  37.57755845,  10.48035311,  18.09386122,
        28.53774704,  15.28447953,  48.73781945,  18.09386122,
        15.28447953,  21.2154998 ,  15.28447953,  42.86394891,
        89.47323891,  42.86394891,  55.26423891,  48.73781945,
        42.86394891,  12.75594298,  24.68388374,  15.28447953,
        24.68388374,  70.57323891,  24.68388374,  37.57755845,
        24.68388374,  12.75594298,  42.86394891,  15.28447953,
        18.09386122,  24.68388374,  18.09386122,  48.73781945,
        48.73781945,  79.52591945,  48.73781945,  55.26423891,
        37.57755845,  10.48035311,  21.2154998 ,  12.75

In [320]:
print(f"Value Iteration took {end_time - start_time:.4f} seconds")

Value Iteration took 0.5883 seconds


In [321]:
#Policy_Iteration
start_time = time.time()
optimal_policy, optimal_p_values = policy_iteration(env, max_iters=500, gamma=0.9)
end_time = time.time()

Converged at 88-th iteration.
Converged at 97-th iteration.
Converged at 100-th iteration.
Converged at 101-th iteration.
Converged at 102-th iteration.
Converged at 103-th iteration.
Converged at 106-th iteration.
Converged at 109-th iteration.
Converged at 110-th iteration.
Converged at 111-th iteration.
Converged at 112-th iteration.
Converged at 115-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Converged at 116-th iteration.
Policy iteration converged at iteration 16.


In [322]:
optimal_p_values

array([ 89.47323891,  32.81971401,  55.26423891,  37.57755845,
         8.43222921,  32.81971401,   8.43222921,  15.28447953,
        32.81971401,  18.09386122,  55.26423891,  21.2154998 ,
        12.75594298,  18.09386122,  12.75594298,  37.57755845,
       100.52591945,  37.57755845,  62.51591945,  42.86394891,
        79.52591945,  28.53774704,  48.73781945,  32.81971401,
        10.48035311,  37.57755845,  10.48035311,  18.09386122,
        28.53774704,  15.28447953,  48.73781945,  18.09386122,
        15.28447953,  21.2154998 ,  15.28447953,  42.86394891,
        89.47323891,  42.86394891,  55.26423891,  48.73781945,
        42.86394891,  12.75594298,  24.68388374,  15.28447953,
        24.68388374,  70.57323891,  24.68388374,  37.57755845,
        24.68388374,  12.75594298,  42.86394891,  15.28447953,
        18.09386122,  24.68388374,  18.09386122,  48.73781945,
        48.73781945,  79.52591945,  48.73781945,  55.26423891,
        37.57755845,  10.48035311,  21.2154998 ,  12.75

In [323]:
print(f"Policy Iteration took {end_time - start_time:.4f} seconds")

Policy Iteration took 1.1399 seconds


## Nhận xét

**Thời gian chạy:**

Value Iteration: 0.5883 seconds

Policy Iteration: 1.1399 seconds

**Nhận xét**

Ở toy games này thuật toán Value Iteration chạy nhanh hơn hẳn vì những lý do sau đây:

*   Taxi-v3 có tới 500 trạng thái và nhiều hành động, làm bước đánh giá chính sách trong Policy Iteration tốn rất nhiều thời gian vì phải tính giá trị trạng thái cho toàn bộ chính sách hiện tại, lặp nhiều lần mới hội tụ.
*   Value Iteration chỉ cập nhật giá trị theo công thức Bellman tối ưu mỗi vòng, không cần đánh giá chính sách đầy đủ, nên mỗi vòng lặp nhanh hơn rất nhiều.








